In [1]:
import sys
sys.path.append("C:/Users/jnevin/Documents/GitHub/networkdiffusionanalyser")

from netdiffanalyse.datahandling.dataintegration import FeatureSetup, MatchClassifierFit, NetworkIntegration
from netdiffanalyse.diffusionmodel.diffusionmodel import CustomDiffusionModel, InitialisedDiffusionModel

In [2]:
import pandas as pd
import numpy as np
import networkx as nx

test_df = pd.read_csv('test_df.csv')
test_df.index = test_df.rec_id
test_df = test_df.drop(columns = ['rec_id'])

test_matches = np.load('test_matches.npy', allow_pickle = True)
test_matches = pd.MultiIndex.from_tuples(test_matches)

test_graph = nx.read_gml('test_graph.gml')

In [3]:
# data integration

In [4]:
test_blocks = {'Block': [['given_name', 'given_name']]}
test_compares = {'Exact':  [['given_name', 'given_name'], ['date_of_birth', 'date_of_birth'],
                          ['suburb', 'suburb'], ['state', 'state']],
                'String': [['surname', 'surname', 'jarowinkler', 0.85], ['address_1', 'address_1', 'levenshtein', 0.85]]}

In [5]:
test_feature_setup = FeatureSetup(test_blocks, test_compares, test_df)

In [6]:
test_feature_setup.initialise_indexer()
test_feature_setup.get_candidate_links()

In [7]:
len(test_feature_setup.candidate_links) # 2082

2082

In [8]:
test_feature_setup.initialise_comparer()
test_feature_setup.calculate_features()

In [9]:
test_feature_setup.features.sum(axis=1).value_counts().sort_index(ascending=False) # 142, 145, 30, 9, 376, 1380

6.0     142
5.0     145
4.0      30
3.0       9
2.0     376
1.0    1380
dtype: int64

In [10]:
test_classifier = MatchClassifierFit('NaiveBayesClassifier', test_feature_setup.features, test_matches)
test_classifier.fit_model()

In [11]:
test_classifier.pred_matches(test_feature_setup.features)

MultiIndex([('rec-122-dup-0',   'rec-122-org'),
            (  'rec-183-org', 'rec-183-dup-0'),
            ('rec-248-dup-0',   'rec-248-org'),
            ('rec-469-dup-0',   'rec-469-org'),
            ('rec-373-dup-0',   'rec-373-org'),
            (   'rec-10-org',  'rec-10-dup-0'),
            ('rec-342-dup-0',   'rec-342-org'),
            (  'rec-397-org', 'rec-397-dup-0'),
            (  'rec-472-org', 'rec-472-dup-0'),
            (  'rec-330-org', 'rec-330-dup-0'),
            ...
            (  'rec-5-dup-0',     'rec-5-org'),
            ('rec-407-dup-0',   'rec-407-org'),
            ('rec-367-dup-0',   'rec-367-org'),
            ('rec-103-dup-0',   'rec-103-org'),
            ('rec-195-dup-0',   'rec-195-org'),
            ('rec-184-dup-0',   'rec-184-org'),
            (  'rec-252-org', 'rec-252-dup-0'),
            ( 'rec-48-dup-0',    'rec-48-org'),
            ('rec-298-dup-0',   'rec-298-org'),
            (  'rec-282-org', 'rec-282-dup-0')],
           names=['rec_

In [12]:
test_network_integrator = NetworkIntegration(test_graph, test_classifier.pred_matches(test_feature_setup.features),
                                             'walktrap_integration')

In [13]:
test_network_integrator.integrate_network()

In [14]:
org_graph = test_network_integrator.graph
adj_graph = test_network_integrator.adj_graph

In [15]:
len(adj_graph.nodes()) # 646

646

In [16]:
len(org_graph.nodes()) # 922

922

In [17]:
# diffusion model

In [18]:
model_name = 'sir'
statuses = ['Susceptible', 'Infected', 'Removed']
compartments = {'NodeStochastic': {'c1': [0.02, 'Infected'], 'c2': [0.01]}}
rules = [["Susceptible", "Infected", "c1"], ["Infected", "Removed", "c2"]]
parameters = [['fraction_infected', 0.1]]

In [19]:
custom_diffusion_model = CustomDiffusionModel(model_name, statuses, compartments,
                                             rules, parameters)

In [20]:
g = nx.erdos_renyi_graph(1000, 0.1)
initialised_diffusion_model = InitialisedDiffusionModel(g, custom_diffusion_model)

In [21]:
initialised_diffusion_model.initialise_model()
initialised_diffusion_model.add_statuses()
initialised_diffusion_model.add_compartments()
initialised_diffusion_model.add_rules()
initialised_diffusion_model.set_initial_model_status()

In [25]:
initialised_diffusion_model.model

In [26]:
import networkx as nx
import ndlib.models.ModelConfig as mc
import ndlib.models.CompositeModel as gc
import ndlib.models.compartments as cpm

# Network generation
g = nx.erdos_renyi_graph(1000, 0.1)

# Composite Model instantiation
model = gc.CompositeModel(g)

# Model statuses
model.add_status("Susceptible")
model.add_status("Infected")
model.add_status("Removed")

# Compartment definition
c1 = cpm.NodeStochastic(0.02, triggering_status="Infected")
c2 = cpm.NodeStochastic(0.01)

# Rule definition
model.add_rule("Susceptible", "Infected", c1)
model.add_rule("Infected", "Removed", c2)

# Model initial status configuration
config = mc.Configuration()
config.add_model_parameter('fraction_infected', 0.1)

# Simulation execution
model.set_initial_status(config)
iterations = model.iteration_bunch(5)

In [27]:
iterations

[{'iteration': 0,
  'status': {0: 1,
   1: 0,
   2: 0,
   3: 0,
   4: 0,
   5: 0,
   6: 0,
   7: 0,
   8: 0,
   9: 0,
   10: 0,
   11: 0,
   12: 1,
   13: 0,
   14: 0,
   15: 0,
   16: 0,
   17: 0,
   18: 0,
   19: 0,
   20: 1,
   21: 0,
   22: 0,
   23: 0,
   24: 1,
   25: 1,
   26: 0,
   27: 0,
   28: 0,
   29: 0,
   30: 0,
   31: 0,
   32: 0,
   33: 1,
   34: 0,
   35: 0,
   36: 0,
   37: 0,
   38: 0,
   39: 0,
   40: 0,
   41: 0,
   42: 0,
   43: 1,
   44: 0,
   45: 0,
   46: 0,
   47: 0,
   48: 0,
   49: 0,
   50: 0,
   51: 0,
   52: 0,
   53: 0,
   54: 0,
   55: 0,
   56: 0,
   57: 0,
   58: 0,
   59: 0,
   60: 0,
   61: 0,
   62: 0,
   63: 0,
   64: 0,
   65: 0,
   66: 0,
   67: 1,
   68: 1,
   69: 0,
   70: 0,
   71: 0,
   72: 0,
   73: 0,
   74: 0,
   75: 0,
   76: 1,
   77: 0,
   78: 0,
   79: 0,
   80: 0,
   81: 0,
   82: 1,
   83: 0,
   84: 0,
   85: 0,
   86: 0,
   87: 1,
   88: 0,
   89: 0,
   90: 0,
   91: 1,
   92: 1,
   93: 0,
   94: 0,
   95: 0,
   96: 1,
   97: 0,
  